In [1]:
import time

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import NoAlertPresentException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import warnings
warnings.simplefilter('ignore')

In [15]:
# chrome driverを作成
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://dominion.games')



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/kazuki/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


### 起動したchromeで自分でログインしてから以下のコードを実行している

In [16]:
def extract_supply(game_id):
    # 新規卓 をクリック
    driver.find_elements_by_class_name("tab-button")[1].click()
    time.sleep(0.125)

    # 古い対戦を読み込む をクリック
    try:
        driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/div/table-starter/div/div[1]/button[2]")[0].click()
        time.sleep(0.25)

        # 対戦IDを入力
        game_id_form = driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/div/table-starter/div/div[2]/input")[0]
        game_id_form.send_keys(game_id)
        time.sleep(0.125)

        # 対戦を読み込む をクリック
        driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/div/table-starter/div/div[2]/button[1]")[0].click()
        time.sleep(0.25)
    except IndexError:
        pass
    
    # AIを加える　をクリック
    driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/rules-editor/div/div[2]/button")[0].click()
    time.sleep(0.125)
    
    # 対戦を読み込む をクリック
    driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/rules-editor/div/div[5]/button")[0].click()
    time.sleep(1)

    # 準備完了 をクリック
    driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/table-buttons/div/button[1]")[0].click()
    time.sleep(5)

    ### ここからゲーム画面
    return extract_supply_in_game_mode()
    
"""
def extract_supply(game_id):    
    # 対戦IDを入力
    game_id_form = driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/rules-editor/div/div[5]/table/tbody/tr[1]/td[1]/input")[0]
    game_id_form.clear()
    game_id_form.send_keys(game_id)
    time.sleep(0.5)

    # 何番目のアクション化を入力
    action_num_form = driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/rules-editor/div/div[5]/table/tbody/tr[2]/td[1]/input")[0]
    action_num_form.clear()
    action_num_form.send_keys("0")
    time.sleep(0.5)

    # 対戦を読み込む をクリック
    driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/rules-editor/div/div[5]/button")[0].click()
    time.sleep(0.5)

    # 準備完了 をクリック
    driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/table-buttons/div/button[1]")[0].click()
    time.sleep(10)

    ### ここからゲーム画面
    return extract_supply_in_game_mode()
"""

def extract_supply_in_game_mode():
    is_finish = False
    supply_cards = set()  # カードが重複する可能性がある
    
    # hex, boonが画面に表示されているときは失敗するので
    # 適当なところを一度クリックして消す
    try:
        elements = driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[6]/top-controls/div/div[1]")[0]
        actions = ActionChains(driver)
        actions.click()
        actions.double_click()
        actions.perform()
        # 王国一覧 をクリック
        elements.click()
    except IndexError:
        time.sleep(20)
        elements = driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[6]/top-controls/div/div[1]")[0]
        actions = ActionChains(driver)
        actions.click()
        actions.perform()
        # 王国一覧 をクリック
        elements.click()

    finish_turn = get_finish_turn()
        
    # 終局しましたボタンが表示されてたらゲーム終了条件を満たしたゲーム
    if len(driver.find_elements_by_xpath("/html/body/div[2]/div/div/modal-game-windows/game-ended-notification/modal-window/div/div/ng-transclude/div")) == 1:
        is_finish = True

    # サプライ一覧取得
    for element in driver.find_elements_by_class_name("mini-card-art"):
        supply_cards.add(element.get_attribute("style").split("/")[-1].split(".")[0].replace("mini-", ""))

    for element in driver.find_elements_by_class_name("full-card-art"):
        supply_cards.add(element.get_attribute("style").split("/")[-1].split(".")[0].replace("mini-", ""))

    for element in driver.find_elements_by_class_name("landscape-art"):
        supply_cards.add(element.get_attribute("style").split("/")[-1].split(".")[0].replace("mini-", ""))

    # 投了ボタン をクリック
    driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[6]/top-controls/div/div[4]")[0].click()
    time.sleep(0.5)
    try:
        Alert(driver).accept()
    except NoAlertPresentException:
        pass
    time.sleep(0.5)
    try:
        driver.find_elements_by_xpath("/html/body/div[2]/div/div/modal-game-windows/game-ended-notification/modal-window/div/div/ng-transclude/button")[0].click()
    except IndexError:
        # 遅れやすいのでもう少し待ってチャレンジ
        time.sleep(5)
        driver.find_elements_by_xpath("/html/body/div[2]/div/div/modal-game-windows/game-ended-notification/modal-window/div/div/ng-transclude/button")[0].click()

    time.sleep(3)
    
    # 卓の編集 をクリック
    try:
        driver.find_elements_by_xpath("/html/body/div[3]/div/div[1]/score-table/score-table-buttons/div/button[2]")[0].click()
        time.sleep(1)
    except IndexError:
        time.sleep(5)
        driver.find_elements_by_xpath("/html/body/div[3]/div/div[1]/score-table/score-table-buttons/div/button[2]")[0].click()

    return supply_cards, finish_turn, is_finish

def get_finish_turn():
    first_player = None
    last_turn_text = None
    last_turn = None

    for log_text in driver.find_elements_by_class_name("log-line")[:10]:
        if "ターン " in log_text.text:
            if first_player is None:
                first_player = log_text.text.split()[-1]

            last_turn_text = log_text.text
            
    for log_text in driver.find_elements_by_class_name("log-line")[::-1]:
        if "ターン " in log_text.text:
            last_turn_text = log_text.text
            break

    # 先手の14ターン目に終わったら 13.5, 後手の14ターン目に終わったら 14.0
    if last_turn_text is not None:
        last_turn = int(last_turn_text.split()[1])
        if first_player is not None:
            if first_player in last_turn_text:
                last_turn -= 0.5
    
    return last_turn    

def delete_basic_cards(supply_cards):
    supply_cards = supply_cards
    
    basic_cards = [
        "copper",
        "silver",
        "gold",
        "estate",
        "duchy",
        "province",
        "curse",
        "potion",
        "will-o-wisp",
        "imp",
        "ghost",
        "deluded",
        "envious",
        "miserable",
        "twice-miserable",
        "bad-omens",
        "delusion",
        "envy",
        "famine",
        "fear",
        "greed",
        "haunting",
        "locusts",
        "misery",
        "plague",
        "poverty",
        "war",
        "spoils",
        "ruined-library",
        "ruined-market",
        "ruined-village",
        "survivors",
        "abandoned-mine",
        "madman",
        'the-earths-gift',
        'the-fields-gift',
        'the-flames-gift',
        'the-forests-gift',
        'the-moons-gift',
        'the-mountains-gift',
        'the-rivers-gift',
        'the-seas-gift',
        'the-skys-gift',
        'the-suns-gift',
        'the-swamps-gift',
        'the-winds-gift',
        "empty",
    ]
    
    for basic_card in basic_cards:
        if basic_card in supply_cards:
            supply_cards.remove(basic_card)
    
    return supply_cards

In [17]:
game_ids = []
with open("../resources/game_id_list.txt") as f:
    for line in f:
        game_ids.append(line.rstrip())
        
# 重複除去
game_ids = list(set(game_ids))

In [21]:
output_dir = "../resources/supplys/"

# 一度情報を集めた対戦はスキップしたい
already_done_supplys = set()

import glob

files = glob.glob(f"{output_dir}/*.txt")
for file in files:
    already_done_supplys.add(file.split("/")[-1].replace(".txt", ""))

In [22]:
len(already_done_supplys)

3245

In [34]:
a,b,_ = extract_supply("90579742")
delete_basic_cards(a), b

({'ambassador',
  'bureaucrat',
  'colony',
  'fortune',
  'gladiator',
  'hideout',
  'hoard',
  'lighthouse',
  'platinum',
  'plaza',
  'spices',
  'transport',
  'venture',
  'werewolf'},
 13)

In [20]:
for i, game_id in enumerate(tqdm(game_ids)): 
    if game_id in already_done_supplys:
        continue
    
    try:
        supply_cards, finish_turn, is_finish = extract_supply(game_id)
    except IndexError:
        # たまに止まるので再起動してやり直し
        driver.refresh()
        time.sleep(30)

        # 投了ボタン をクリック
        try:
            elements = driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[6]/top-controls/div/div[4]")
            if elements:
                elements[0].click()
                time.sleep(0.5)
                Alert(driver).accept()
                time.sleep(5)
                driver.find_elements_by_xpath("/html/body/div[2]/div/div/modal-game-windows/game-ended-notification/modal-window/div/div/ng-transclude/button")[0].click()
                time.sleep(3)
        except UnexpectedAlertPresentException:
            # gameidが読み込めないときはもとに戻る
            #Alert(driver).accept()
            actions = ActionChains(driver)
            actions.double_click()
            actions.perform()
            time.sleep(10)
            continue
        
        # 退室を押す
        elements = driver.find_elements_by_xpath("/html/body/div[3]/div/div[1]/score-table/score-table-buttons/div/button[2]")
        if elements:
            elements[0].click()
            
        time.sleep(5)
        continue
    except UnexpectedAlertPresentException:
        # gameidが読み込めないときはもとに戻る
        driver.find_elements_by_xpath("/html/body/div[2]/div/div/ul/li[1]/button")[0].click()
        time.sleep(1)
        continue
        
    if len(supply_cards) == 0:
        continue
        
    supply_cards = delete_basic_cards(supply_cards)
    
    with open(f"{output_dir}/{game_id}.txt", "w") as f:
        supply_cards_str = " ".join(supply_cards)
        f.write(f"{supply_cards_str},{finish_turn},{is_finish}\n")
        
    time.sleep(10)
    already_done_supplys.add(game_id)

 11%|███████████████▌                                                                                                                        | 796/6982 [42:47<5:32:36,  3.23s/it]


UnexpectedAlertPresentException: Alert Text: 内部エラーです。
Message: unexpected alert open: {Alert text : 内部エラーです。}
  (Session info: chrome=96.0.4664.93)
Stacktrace:
0   chromedriver                        0x000000010143b269 __gxx_personality_v0 + 582729
1   chromedriver                        0x00000001013c6c33 __gxx_personality_v0 + 106003
2   chromedriver                        0x0000000100f83e28 chromedriver + 171560
3   chromedriver                        0x0000000100fe925e chromedriver + 586334
4   chromedriver                        0x0000000100fd6c23 chromedriver + 511011
5   chromedriver                        0x0000000100fac75e chromedriver + 337758
6   chromedriver                        0x0000000100fada95 chromedriver + 342677
7   chromedriver                        0x00000001013f78ab __gxx_personality_v0 + 305803
8   chromedriver                        0x000000010140e863 __gxx_personality_v0 + 399939
9   chromedriver                        0x0000000101413c7f __gxx_personality_v0

In [33]:
game_id = "90575489"
# 新規卓 をクリック
driver.find_elements_by_class_name("tab-button")[1].click()
time.sleep(0.25)

# 古い対戦を読み込む をクリック
driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/div/table-starter/div/div[1]/button[2]")[0].click()
time.sleep(0.5)

# 対戦IDを入力
game_id_form = driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/div/table-starter/div/div[2]/input")[0]
game_id_form.send_keys(game_id)
time.sleep(0.25)

# 対戦を読み込む をクリック
driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/div/table-starter/div/div[2]/button[1]")[0].click()
time.sleep(0.5)

# AIを加える　をクリック
driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/rules-editor/div/div[2]/button")[0].click()
time.sleep(0.25)

# 対戦を読み込む をクリック
driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/rules-editor/div/div[5]/button")[0].click()
time.sleep(0.25)

# 準備完了 をクリック
driver.find_elements_by_xpath("/html/body/div[2]/div/div/div[1]/div/my-table/table-buttons/div/button[1]")[0].click()
time.sleep(3)

In [40]:
driver.quit()